# Pipline

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer

data = pd.read_excel("OMSK-PKN_GP1.1_Фасады.xlsx", sheet_name="Фасады_Секции по файлу")
data

#=== Clean data
df = data[['Unnamed: 2', 'Unnamed: 3']]
df = df.dropna()

df = df[df['Unnamed: 2'].str.count(r"\.") > 1]

df.rename(columns={'Unnamed: 2': "Position", "Unnamed: 3":"ElementId"}, inplace=True)
df = df.reset_index(drop=True)
df['ElementId'] = df['ElementId'].astype('int64')
tangle_data = df

#==== Concat all csv
import glob
def multi_concat():
    csv_files = glob.glob('*.csv')
    out_df = pd.DataFrame()
    
    for file in csv_files:
        if "revit_params_OM" in file:
            df = pd.read_csv(file)
            
            if out_df.empty:
                out_df = df
            else:
                out_df = pd.concat([out_df, df], sort=False, axis=0)
    
    return out_df

revit_data = multi_concat()

#=== Marge
df_groups = df.groupby('ElementId')['Position'].apply(list).reset_index()
marged_df = df_groups.merge(revit_data, left_on='ElementId', right_on='ElementId')


C:\Users\homer\AppData\Local\Temp\ipykernel_16416\2993012231.py:29: DtypeWarning: Columns (102,115,118,121,123,125,142,143,156,164,167,170,175,177,183,184,185,187,188,189,190,192,198,200,202,208,215,216,220,244,285,292,324,325,339,414,418,420,421,445,462,464,511,512,513,582,602,603,606,607,674,684,685,703,704,705,706,707,708,709,710,711,712,713,714,718,719,720,721,722,723,734,738,739,743,744,745,746,747,751,752,753,754,755,756,757,758,792,793,795,821,822,854,858,880,891,892,893,894,897,898,909,910,911,912,913,914,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,949,950,952,953,954,955,956,957,958,959,990,1015,1041,1053,1062,1073,1074,1075,1076,1083,1084,1085,1086,1087,1088,1089,1090,1099,1100,1101,1111,1118,1128,1157,1159,1211,1212,1219,1220,1268,1307,1315,1318,1354,1355,1356,1357,1408,1409,1410,1442,1488,1491,1508,1509,1513,1557,1558,1565,1571,1573,1575,1578,1581,1583,1585,1586,1593,1596,1607,1608,1638,1641

The data is imbalance. <br>
I can remember about it.

In [2]:
#=== marged_df split and clear
target_cols = ['ElementId', 'Position']
X_df = marged_df.drop(columns=target_cols, axis=1)
to_drop = ['Марка', 'Граница помещения','Комментарии','Линия привязки','Маркировка типоразмера','ADSK_Этаж', 'Цвет штриховки при низкой детализации']
X_df.drop(columns=to_drop, inplace=True)
X_df['Зависимость снизу'] = X_df['Зависимость снизу'].str.split('(', expand=True)[0]
X_df['Зависимость сверху'] = X_df['Зависимость сверху'].str.split('(', expand=True)[0]
X_df['Уровень'] = X_df['Уровень'].str.split('(', expand=True)[0]

y_df = marged_df[target_cols]

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
fit_transform = mlb.fit_transform(y_df['Position'])
y_df = pd.DataFrame(fit_transform, columns=mlb.classes_,index=y_df['ElementId'])
y_df = y_df.add_prefix('Target_')

constant_cols = X_df.columns[X_df.nunique() <= 1]
X_df = X_df.drop(columns=constant_cols)
print("Константные колонки:", len(list(constant_cols)))

# X_df.fillna("NAN", inplace=True)

Константные колонки: 2267


In [54]:
X_df['Штриховка при низкой детализации'].value_counts()

Штриховка при низкой детализации
<Сплошная заливка>                 3495
ADSK_Линия_Накрест косая_1x1 мм     143
Name: count, dtype: int64

In [64]:
X_df.columns

Index(['ADSK_Группирование', 'ADSK_Зона', 'ADSK_Номер секции',
       'ADSK_Основной материал элемента', 'ADSK_Примечание',
       'ADSK_Размер_Высота', 'ADSK_Размер_Толщина', 'ADSK_Размер_Ширина',
       'Category', 'PS_Секция_I', 'PS_Секция_II', 'PS_Секция_III',
       'PS_Секция_IV', 'PS_Секция_IX', 'PS_Секция_V', 'PS_Секция_VI',
       'PS_Секция_VII', 'PS_Секция_VIII', 'PS_Секция_Число',
       'Включить аналитическую модель', 'Высота верхнего бруса',
       'Высота нижнего бруса', 'Зависимость сверху', 'Зависимость снизу',
       'Материал отделки основной', 'Материал откоса верхний',
       'Материал откоса левый', 'Материал откоса нижний',
       'Материал откоса правый', 'Неприсоединенная высота',
       'Несущие конструкции', 'Огибание в местах вставки элементов',
       'Огибание в торцах стен', 'Описание', 'Откос верхний', 'Откос нижний',
       'Рабочий набор', 'Семейство', 'Семейство и типоразмер',
       'Смещение от уровня', 'Смещение сверху', 'Смещение снизу',
       '

## Обучение

In [4]:


# lgbm = LGBMClassifier(
#     random_state=42,
#     num_leaves=10,
#     max_depth=5,
#     device='gpu',
#     use_missing=True, 
#     zero_as_missing=False,
#     class_weight='balanced',
#     subsample=0.8,
#     colsample_bytree=0.8,
#     reg_alpha=0.1,
#     reg_lambda=0.1,
#     min_child_samples=20,
# )


# Ручное обучение через цикл. На каждый таргет своя модель. + баланс классов
# multi_lgbm = []
# for i in range(y_df.shape[1]):
#     y_i = y_df.iloc[:, i]
#     lgbm_c = lgbm
#     lgbm_c.fit(X_df, y_i)
#     multi_lgbm.append(lgbm_c)

# joblib.dump(multi_lgbm, 'lgbm.pkl')

# multi_lgbm = MultiOutputClassifier(lgbm)


# for col in cat_features:
#     try:
#         # test_revit_data[col] = test_revit_data[col].astype(str)
#         """
#         Чести параметров что были на обучении может не быть. Если такое произайдет, то скорее
#         всего такое параметр не сильно влияет. Но что бы все работало корректно, такие параметры 
#         заполняются специальным пропуском. 
#         """
#         test_revit_data[col] = test_revit_data[col].astype(str).astype('category')
#     except KeyError:
#         print(f"Параметр '{col}' не найден в данных и будет заполнен как 'missing'")
#         test_revit_data[col] = 'missing'
#         # test_cat_features.remove(col)
#         continue  # или ничего не делать
from sklearn.multioutput import MultiOutputClassifier
from sklearn.utils.class_weight import compute_sample_weight
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import joblib
import json
cat_features = []
for col in X_df.select_dtypes('object').columns:
    X_df[col] = X_df[col].astype(str)
    cat_features.append(col)
    X_df[col] = X_df[col].astype('category')

catb = CatBoostClassifier(
    random_state=42,
    nan_mode='Max',           # или 'Max' — попробуй оба
    max_depth=6,              # можно увеличить до 6–8
    learning_rate=0.1,        # стандартное значение
    n_estimators=200,         # количество деревьев
    verbose=100,              # лог каждые 100 итераций
    scale_pos_weight=None,    # мы сами задаём class_weights
    cat_features=cat_features,# можно вынести, но ниже мы передаём в fit
    eval_metric='F1',         # если важна балансировка, F1 лучше accuracy
    auto_class_weights='Balanced',  # ! Отличная фича: автоматически весит классы
)

# CatBoost - Ручное обучение через цикл. На каждый таргет своя модель. + баланс классов
multi_catb = []
for i in range(y_df.shape[1]):
    y_i = y_df.iloc[:, i]
    # Вычисляем веса классов
    # unique, counts = np.unique(y_i, return_counts=True)
    # neg_weight = counts[1] / counts[0] if counts[0] != 0 else 1.0
    # class_weights = [neg_weight, 1.0]  # [вес для 0, вес для 1])
    cbc = CatBoostClassifier(**catb.get_params())
    cbc.fit(X_df, y_i)

    multi_catb.append(cbc)

joblib.dump(multi_catb, 'catb.pkl')

# multi_catb = MultiOutputClassifier(catb)
# #=== обучение
# print("Обучение LightGBM")
# multi_lgbm.fit(X_df, y_df)

# print("Обучение CatBoost")
# multi_catb.fit(X_df, y_df, cat_features=cat_features)


0:	learn: 0.9367506	total: 179ms	remaining: 35.6s
100:	learn: 0.9992551	total: 2.85s	remaining: 2.79s
199:	learn: 0.9998768	total: 5.62s	remaining: 0us
0:	learn: 0.9585656	total: 24.5ms	remaining: 4.87s
100:	learn: 0.9717930	total: 2.77s	remaining: 2.72s
199:	learn: 0.9731523	total: 5.59s	remaining: 0us
0:	learn: 0.9585656	total: 24.1ms	remaining: 4.8s
100:	learn: 0.9717930	total: 2.77s	remaining: 2.71s
199:	learn: 0.9731523	total: 5.55s	remaining: 0us
0:	learn: 0.9585656	total: 36.8ms	remaining: 7.32s
100:	learn: 0.9717930	total: 2.71s	remaining: 2.66s
199:	learn: 0.9731523	total: 5.56s	remaining: 0us
0:	learn: 0.9585656	total: 23.6ms	remaining: 4.69s
100:	learn: 0.9717930	total: 2.75s	remaining: 2.69s
199:	learn: 0.9731523	total: 5.59s	remaining: 0us
0:	learn: 0.9638496	total: 30.4ms	remaining: 6.04s
100:	learn: 0.9847424	total: 2.71s	remaining: 2.65s
199:	learn: 0.9853585	total: 5.5s	remaining: 0us
0:	learn: 0.9638496	total: 52.6ms	remaining: 10.5s
100:	learn: 0.9847424	total: 2.72s

['catb.pkl']

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import joblib
import json

lgbm = LGBMClassifier(
    random_state=42,
    num_leaves=10,
    max_depth=5,
    device='gpu',
    use_missing=True, 
    zero_as_missing=False,
    class_weight='balanced',
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    min_child_samples=20,
)

catb = CatBoostClassifier(
    random_state=42,
    nan_mode="Min",
    max_depth=5
)

# multi_lgbm = MultiOutputClassifier(lgbm)

cat_features = []
for col in X_df.select_dtypes('object').columns:
    X_df[col] = X_df[col].astype(str)
    cat_features.append(col)
    X_df[col] = X_df[col].astype('category')

# for col in cat_features:
#     try:
#         # test_revit_data[col] = test_revit_data[col].astype(str)
#         """
#         Чести параметров что были на обучении может не быть. Если такое произайдет, то скорее
#         всего такое параметр не сильно влияет. Но что бы все работало корректно, такие параметры 
#         заполняются специальным пропуском. 
#         """
#         test_revit_data[col] = test_revit_data[col].astype(str).astype('category')
#     except KeyError:
#         print(f"Параметр '{col}' не найден в данных и будет заполнен как 'missing'")
#         test_revit_data[col] = 'missing'
#         # test_cat_features.remove(col)
#         continue  # или ничего не делать

multi_catb = MultiOutputClassifier(catb)

#=== обучение
print("Обучение LightGBM")
multi_lgbm.fit(X_df, y_df)

print("Обучение CatBoost")
multi_catb.fit(X_df, y_df, cat_features=cat_features)


Обучение LightGBM
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1332, number of negative: 4059
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 657
[LightGBM] [Info] Number of data points in the train set: 5391, number of used features: 60
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3080, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 24 dense feature groups (0.12 MB) transferred to GPU in 0.002210 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

MultiOutputClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x00000197945BB050>)

In [4]:
import joblib
import json

# Сохранение моделей, cat_features и фичей
joblib.dump(multi_lgbm, 'lgbm.pkl')
joblib.dump(multi_catb, 'catb.pkl')

with open('features.json', 'w') as f:
    json.dump(X_df.columns.to_list(), f)

print("Модели и метаданные сохранены.")

Модели и метаданные сохранены.


In [5]:
with open('trained_targets.json', 'w') as f:
    json.dump(y_df.columns.to_list(), f)

In [6]:
with open('cat_features.json', 'w') as f:
    json.dump(cat_features, f)

In [7]:
with open('features.json', 'r') as f:
    features = json.load(f)


In [9]:
from sklearn.metrics import accuracy_score, hamming_loss, f1_score

#=== Предсказание вероятностей
# y_pred_proba_lgbm = np.array([clf.predict_proba(X_df)[:, 1] for clf in multi_lgbm.estimators_]).T
y_pred_proba_ensemble = np.array([clf.predict_proba(X_df)[:, 1] for clf in multi_catb]).T

# y_pred_proba_ensemble = (y_pred_proba_lgbm + y_pred_proba_catb) / 2

#=== отсечения порогом вероятности 
y_pred_ensemble = (y_pred_proba_ensemble > 0.5).astype(int)

In [10]:
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score, recall_score

#=== Точность по объектам (все метки совпали)
exact_match = accuracy_score(y_df, y_pred_ensemble, normalize=True)

#=== Hamming Accuracy (доля правильно предсказанных меток)
hamming_acc = 1 - hamming_loss(y_df, y_pred_ensemble)

#=== F1, Precision, Recall — с разными типами усреднения
f1_macro = f1_score(y_df, y_pred_ensemble, average='macro')
f1_micro = f1_score(y_df, y_pred_ensemble, average='micro')

precision_macro = precision_score(y_df, y_pred_ensemble, average='macro', zero_division=0)
precision_micro = precision_score(y_df, y_pred_ensemble, average='micro', zero_division=0)

recall_macro = recall_score(y_df, y_pred_ensemble, average='macro', zero_division=0)
recall_micro = recall_score(y_df, y_pred_ensemble, average='micro', zero_division=0)

#=== Метрики
print("Результаты ансамбля LGBM + CatBoost:")
print(f"Macro F1 Score:       {f1_macro:.4f}")
print(f"Micro F1 Score:       {f1_micro:.4f}")
print(f"Macro Precision:      {precision_macro:.4f}")
print(f"Micro Precision:      {precision_micro:.4f}")
print(f"Macro Recall:         {recall_macro:.4f}")
print(f"Micro Recall:         {recall_micro:.4f}")

Результаты ансамбля LGBM + CatBoost:
Macro F1 Score:       0.9751
Micro F1 Score:       0.9725
Macro Precision:      0.9644
Micro Precision:      0.9685
Macro Recall:         0.9904
Micro Recall:         0.9765


# Проверка на тесовых данных

Интерпретация результатов 

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.preprocessing import MultiLabelBinarizer

test_data = pd.read_excel("KRGN-MK_GP07_Фасады.xlsx", sheet_name="Фасады_Секции по файлу")

#=== Clean data
test_tangle_data = test_data[['Unnamed: 2', 'Unnamed: 3']]
test_tangle_data = test_tangle_data.dropna()

test_tangle_data = test_tangle_data[test_tangle_data['Unnamed: 2'].str.count(r"\.") > 1]

test_tangle_data.rename(columns={'Unnamed: 2': "Position", "Unnamed: 3":"ElementId"}, inplace=True)
test_tangle_data = test_tangle_data.reset_index(drop=True)
test_tangle_data['ElementId'] = test_tangle_data['ElementId'].astype('int64')

tangle_groups = test_tangle_data.groupby('ElementId')['Position'].apply(list).reset_index()
#==== Concat all csv
import glob
def multi_concat():
    csv_files = glob.glob('*.csv')
    out_df = pd.DataFrame()
    
    for file in csv_files:
        if "revit_params_KRGN" in file:
            df = pd.read_csv(file)
            
            if out_df.empty:
                out_df = df
            else:
                out_df = pd.concat([out_df, df], sort=False, axis=0)
    
    return out_df

test_revit_data = multi_concat()

"""
по началу я не хотел марджить, и по идее в проде это тоже не понадобится, но что бы сделать 
замеры метрик видимо нужно мерджить, что бы выровнять датасеты. Иначе обьекты и метки будут в разных позициях
что приведет к неправильному расчету метрик
"""



C:\Users\homer\AppData\Local\Temp\ipykernel_16416\2774325336.py:29: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,184,185,186,189,194,195,197,198,199,201,203,205,207,208,210,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,2

'\nпо началу я не хотел марджить, и по идее в проде это тоже не понадобится, но что бы сделать \nзамеры метрик видимо нужно мерджить, что бы выровнять датасеты. Иначе обьекты и метки будут в разных позициях\nчто приведет к неправильному расчету метрик\n'

In [15]:
#=== Marge
# tangle_groups = tangle_groups.sample(frac=1, random_state=42).reset_index()
# del tangle_groups['index']
test_marged_df = tangle_groups.merge(test_revit_data, left_on='ElementId', right_on='ElementId')
# test_marged_df = test_marged_df.sample(frac=1, random_state=42).reset_index()

~~Преобразования датафреймов. y - tangle_groups, x - test_revit_data~~

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer
import json

#=== marged_df split and clear
target_cols = ['ElementId', 'Position']
test_ids = test_marged_df['ElementId'].copy()
X = test_marged_df.drop(columns=target_cols, axis=1)
to_drop = ['Марка', 'Граница помещения','Комментарии','Линия привязки','Маркировка типоразмера','ADSK_Этаж', 'Цвет штриховки при низкой детализации']
X.drop(columns=to_drop, inplace=True)
X['Зависимость снизу'] = X['Зависимость снизу'].str.split('(', expand=True)[0]
X['Зависимость сверху'] = X['Зависимость сверху'].str.split('(', expand=True)[0]
X['Уровень'] = X['Уровень'].str.split('(', expand=True)[0]

y = test_marged_df[target_cols]

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_fit = mlb.fit_transform(y['Position'])
y = pd.DataFrame(y_fit, columns=mlb.classes_,index=y['ElementId'])
y = y.add_prefix('Target_')

with open('features.json', 'r') as f:
    features = json.load(f)

#=== Выбор только нужных фичей
X = X[X.columns.intersection(features)]

# constant_cols = X.columns[X.nunique() <= 1]
# X = X.drop(columns=constant_cols)
# print("Константные колонки:", len(list(constant_cols)))

X.fillna("NAN", inplace=True)

In [20]:
"""
Получилось так, что на тесте больше таргетов, чем на обучении. 
Надо занулить лишние таргеты, что бы занулить вероятности при обучении.
"""
y.columns.__len__() # 31 таргеты на тестовых данных 
y.columns.__len__() # 29 таргеты на обучающих данных 


37

### Убрал MultiOutputClassifie, что бы получить более кнтролируемое качество. 

In [21]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.utils.class_weight import compute_sample_weight
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import joblib
import json

with open('cat_features.json', 'r') as f:
    test_cat_features = json.load(f)

for col in test_cat_features:
    try:
        X[col] = X[col].astype(str)
        """
        Чести параметров что были на обучении может не быть. Если такое произайдет, то скорее
        всего такое параметр не сильно влияет. Но что бы все работало корректно, такие параметры 
        заполняются специальным пропуском. 
        """
        X[col] = X[col].astype('category')
    except KeyError:
        print(f"Параметр '{col}' не найден в данных и будет заполнен как 'missing'")
        X[col] = 'missing'
        # test_cat_features.remove(col)
        continue  # или ничего не делать

catb = CatBoostClassifier(
    random_state=42,
    nan_mode='Max',           # или 'Max' — попробуй оба
    max_depth=6,              # можно увеличить до 6–8
    learning_rate=0.1,        # стандартное значение
    n_estimators=200,         # количество деревьев
    verbose=100,              # лог каждые 100 итераций
    scale_pos_weight=None,    # мы сами задаём class_weights
    cat_features=test_cat_features,# можно вынести, но ниже мы передаём в fit
    eval_metric='F1',         # если важна балансировка, F1 лучше accuracy
    auto_class_weights='Balanced',  # ! Отличная фича: автоматически весит классы
)


#=== Загрузка
print("Загрузка CatBoost завершена")
# multi_lgbm = joblib.load('lgbm.pkl')
multi_catb = joblib.load('catb.pkl')

Загрузка CatBoost завершена


In [34]:
from sklearn.multioutput import MultiOutputClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import joblib

lgbm = LGBMClassifier(
    random_state=42,
    num_leaves=10,
    max_depth=5,
    device='gpu',
    use_missing=True, 
    zero_as_missing=False,

    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    min_child_samples=20,
)

catb = CatBoostClassifier(
    random_state=42,
    nan_mode="Min",
    max_depth=5
)

# multi_lgbm = MultiOutputClassifier(lgbm)

with open('cat_features.json', 'r') as f:
    test_cat_features = json.load(f)

for col in test_cat_features:
    try:
        X[col] = X[col].astype(str)
        """
        Чести параметров что были на обучении может не быть. Если такое произайдет, то скорее
        всего такое параметр не сильно влияет. Но что бы все работало корректно, такие параметры 
        заполняются специальным пропуском. 
        """
        X[col] = X[col].astype('category')
    except KeyError:
        print(f"Параметр '{col}' не найден в данных и будет заполнен как 'missing'")
        X[col] = 'missing'
        # test_cat_features.remove(col)
        continue  # или ничего не делать

multi_catb = MultiOutputClassifier(catb)

#=== Загрузка
print("Загрузка LightGBM и CatBoost завершена")
# multi_lgbm = joblib.load('lgbm.pkl')
multi_catb = joblib.load('catb.pkl')


Параметр 'Марка' не найден в данных и будет заполнен как 'missing'
Загрузка LightGBM и CatBoost завершена


In [22]:
# Предсказание вероятностей
# y_pred_proba_lgbm = np.array([clf.predict_proba(X)[:,1] for clf in multi_lgbm.estimators_]).T
y_pred_proba_ctb = np.array([clf.predict_proba(X)[:,1] for clf in multi_catb]).T

# усреднение вероятностей
# y_pred_proba_ensemble = (y_pred_proba_lgbm + y_pred_proba_catb) / 2

# отсечения порогом вероятности 
y_pred_ensemble = (y_pred_proba_ctb > 0.5).astype(int)

Логика в том что берем только те таргеты которые были в обучении

In [23]:
"""
Получилось так, что на тесте больше таргетов, чем на обучении. y_pred_ensemble.shape - (220244, 29)
Надо занулить лишние таргеты, что бы занулить то, что не предсказано, что бы не ломать метрики. но
У меня сейчас супер мало данных. Так что придется проверять только то что есть. 
"""
y.columns.__len__() # - 37 таргеты на тестовых данных 
y_df.columns.__len__() # - 25 таргеты на обучающих данных 

expected_targets = y.columns.tolist()
with open('trained_targets.json', 'r') as f:
    trained_targets = json.load(f)
trained_targets
print(expected_targets)
print(trained_targets)

indexes = []
data_targets = []
for ind, targ in enumerate(trained_targets):
    if targ in expected_targets :
        print(targ)
        indexes.append(ind)
        data_targets.append(targ)

new_pred = y_pred_ensemble[:, indexes]
new_y = y[data_targets]
# new_y
new_y

['Target_004.014.033.001.002', 'Target_004.014.033.001.005', 'Target_004.014.033.001.008', 'Target_004.014.033.001.012', 'Target_004.014.033.001.013', 'Target_004.014.033.001.015', 'Target_004.014.033.001.3', 'Target_004.014.033.002.001', 'Target_004.014.033.002.002', 'Target_004.014.033.002.003', 'Target_004.014.033.002.004', 'Target_004.014.033.002.005', 'Target_004.014.033.004.001', 'Target_004.014.033.004.002', 'Target_004.014.033.004.003', 'Target_004.014.033.004.004', 'Target_004.014.033.004.005', 'Target_004.014.033.004.006', 'Target_004.014.033.006.001.1', 'Target_004.014.033.006.002', 'Target_021.004.003', 'Target_021.005.003.005.073', 'Target_021.005.003.005.074', 'Target_021.005.003.005.075', 'Target_021.005.003.005.082', 'Target_021.015.001.017', 'Target_031.003.002.006', 'Target_031.003.003.001', 'Target_031.003.003.002', 'Target_031.003.003.003', 'Target_031.003.003.004', 'Target_031.003.003.005', 'Target_031.006.003', 'Target_031.006.004', 'Target_031.006.006', 'Target_0

,Target_004.014.033.001.002,Target_004.014.033.001.005,Target_004.014.033.001.008,Target_004.014.033.001.012,Target_004.014.033.001.013,Target_004.014.033.002.001,Target_004.014.033.002.002,Target_004.014.033.002.003,Target_004.014.033.002.004,Target_004.014.033.002.005,Target_004.014.033.004.001,Target_004.014.033.004.002,Target_004.014.033.004.003,Target_004.014.033.004.004,Target_004.014.033.004.005,Target_004.014.033.004.006,Target_004.014.033.006.001.1,Target_021.015.001.017,Target_031.006.003,Target_031.006.006
ElementId,,,,,,,,,,,,,,,,,,,,
1128759,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1128759,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1128760,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1128760,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1128761,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4692904,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4692909,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5211300,0,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [24]:
# тут сравнения по 2-м датафреймам
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score, recall_score
#=== Точность по объектам (все метки совпали)
exact_match = accuracy_score(new_y,new_pred, normalize=True)

#=== Hamming Accuracy (доля правильно предсказанных меток)
hamming_acc = 1 - hamming_loss(new_y, new_pred)

#=== F1, Precision, Recall — с разными типами усреднения
f1_macro = f1_score(new_y, new_pred, average='macro')
f1_micro = f1_score(new_y, new_pred, average='micro')

precision_macro = precision_score(new_y, new_pred, average='macro', zero_division=0)
precision_micro = precision_score(new_y, new_pred, average='micro', zero_division=0)

recall_macro = recall_score(new_y, new_pred, average='macro', zero_division=0)
recall_micro = recall_score(new_y, new_pred, average='micro', zero_division=0)

#=== Метрики
# print("Результаты ансамбля на тестовых данных LGBM + CatBoost:")
print("Результаты ансамбля LGBM + CatBoost на тестовых данных:")
print(f"Macro F1 Score:       {f1_macro:.4f}")
print(f"Micro F1 Score:       {f1_micro:.4f}")
print(f"Macro Precision:      {precision_macro:.4f}")
print(f"Micro Precision:      {precision_micro:.4f}")
print(f"Macro Recall:         {recall_macro:.4f}")
print(f"Micro Recall:         {recall_micro:.4f}")

Результаты ансамбля LGBM + CatBoost на тестовых данных:
Macro F1 Score:       0.6059
Micro F1 Score:       0.6059
Macro Precision:      0.6550
Micro Precision:      0.6130
Macro Recall:         0.6516
Micro Recall:         0.5991
